# 1. PyMuPDF

In [2]:
! pip install pymupdf pillow

In [4]:
!pip uninstall fitz


^C


In [ ]:
!pip install PyMuPDF

In [ ]:
! import fitz
print(fitz.__doc__)


In [5]:
# import fitz  # PyMuPDF
import pymupdf 
import io
from PIL import Image

In [6]:
# Open the PDF file
pdf_document = pymupdf.open("sample.pdf")

FileNotFoundError: no such file: 'sample.pdf'

In [ ]:

# Iterate over each page
for page_number in range(len(pdf_document)):
    page = pdf_document[page_number]
    image_list = page.get_images(full=True)
    
    # Iterate over each image
    for image_index, img in enumerate(image_list, start=1):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        
        # Load image with PIL
        image = Image.open(io.BytesIO(image_bytes))
        
        # Save image
        image.save(f"page{page_number+1}_image{image_index}.{image_ext}")
